In [9]:
# ======================================================
# ADVANCED SALES ANALYTICS & BILLING SYSTEM
# Python + SQLite + Pandas + Matplotlib
# ======================================================

import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime


# ------------------------------------------------------
# DATABASE CONNECTION
# ------------------------------------------------------
conn = sqlite3.connect("sales_data.db")
cursor = conn.cursor()


# ------------------------------------------------------
# CREATE TABLE
# ------------------------------------------------------
cursor.execute("""
CREATE TABLE IF NOT EXISTS sales (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product TEXT,
    region TEXT,
    quantity INTEGER,
    price REAL,
    cost REAL,
    sale_date TEXT
)
""")
conn.commit()


# ------------------------------------------------------
# FUNCTIONS
# ------------------------------------------------------

# Insert
def insert_data():
    n = int(input("How many records: "))

    for i in range(n):
        print(f"\nRecord {i+1}")
        product = input("Product Name: ")
        region = input("Region: ")
        quantity = int(input("Quantity: "))
        price = float(input("Selling Price: "))
        cost = float(input("Cost Price: "))
        date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        if quantity <= 0 or price <= 0:
            print("❌ Invalid data")
            continue

        cursor.execute("""
        INSERT INTO sales (product, region, quantity, price, cost, sale_date)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (product, region, quantity, price, cost, date))

    conn.commit()
    print("✅ Inserted Successfully!")


# View
def view_data():
    df = pd.read_sql_query("SELECT * FROM sales", conn)
    print("\n📋 Records:")
    print(df)


# Delete
def delete_data():
    id_delete = int(input("Enter ID to delete: "))
    cursor.execute("DELETE FROM sales WHERE id=?", (id_delete,))
    conn.commit()
    print("✅ Deleted!")


# Update
def update_data():
    id_update = int(input("Enter ID to update: "))
    new_qty = int(input("New Quantity: "))
    new_price = float(input("New Selling Price: "))

    cursor.execute("""
    UPDATE sales
    SET quantity=?, price=?
    WHERE id=?
    """, (new_qty, new_price, id_update))

    conn.commit()
    print("✅ Updated Successfully!")


# Search
def search_data():
    region = input("Enter region to search: ")
    df = pd.read_sql_query(
        "SELECT * FROM sales WHERE region=?", conn, params=(region,))
    print(df)


# Analysis
def analyze_data():
    df = pd.read_sql_query("SELECT * FROM sales", conn)

    if df.empty:
        print("⚠ No data")
        return

    df["total_sales"] = df["quantity"] * df["price"]
    df["profit"] = (df["price"] - df["cost"]) * df["quantity"]

    print("\n📊 Total Revenue:", df["total_sales"].sum())
    print("📊 Total Profit:", df["profit"].sum())

    region_sales = df.groupby("region")["total_sales"].sum()

    # Bar chart
    plt.figure()
    region_sales.plot(kind="bar")
    plt.title("Sales by Region")
    plt.xlabel("Region")
    plt.ylabel("Sales")
    plt.show()

    # Pie chart
    plt.figure()
    region_sales.plot(kind="pie", autopct="%1.1f%%", startangle=90)
    plt.title("Region Contribution")
    plt.ylabel("")
    plt.show()

    # Auto insights
    top_product = df.groupby("product")["quantity"].sum().idxmax()
    print("🔥 Top Selling Product:", top_product)


# ------------------------------------------------------
# BILL GENERATION
# ------------------------------------------------------
def generate_bill():
    id_bill = int(input("Enter ID for bill: "))

    df = pd.read_sql_query(
        "SELECT * FROM sales WHERE id=?", conn, params=(id_bill,))

    if df.empty:
        print("❌ Record not found")
        return

    row = df.iloc[0]
    total = row["quantity"] * row["price"]

    print("\n========== PAYMENT BILL ==========")
    print("Bill No:", row["id"])
    print("Date:", row["sale_date"])
    print("Product:", row["product"])
    print("Region:", row["region"])
    print("Quantity:", row["quantity"])
    print("Price:", row["price"])
    print("----------------------------------")
    print("TOTAL AMOUNT:", total)
    print("==================================")


# ------------------------------------------------------
# MENU
# ------------------------------------------------------
while True:
    print("\n=========== SYSTEM MENU ===========")
    print("1. Insert")
    print("2. View")
    print("3. Update")
    print("4. Delete")
    print("5. Search by Region")
    print("6. Analyze & Charts")
    print("7. Generate Bill")
    print("8. Exit")

    choice = input("Enter choice: ")

    if choice == "1":
        insert_data()
    elif choice == "2":
        view_data()
    elif choice == "3":
        update_data()
    elif choice == "4":
        delete_data()
    elif choice == "5":
        search_data()
    elif choice == "6":
        analyze_data()
    elif choice == "7":
        generate_bill()
    elif choice == "8":
        print("👋 Thank you")
        break
    else:
        print("❌ Invalid choice")


# ------------------------------------------------------
# CLOSE
# ------------------------------------------------------
conn.close()



=========== SYSTEM MENU ===========
1. Insert
2. View
3. Update
4. Delete
5. Search by Region
6. Analyze & Charts
7. Generate Bill
8. Exit
Enter choice: 8
👋 Thank you
